We'll be starting off by reading the dataset files from MovieLens using the 1M dataset.

https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/

https://alyssaq.github.io/2015/20150426-simple-movie-recommender-using-svd/

In [7]:
import pandas as pd
import numpy as np

ratings_data = pd.io.parsers.read_csv('/data/ml-1m/ratings.dat', 
        names=['userID','movieID','rating','timestamp'], 
        engine='python', delimiter='::')
movies_df = pd.io.parsers.read_csv('/data/ml-1m/ratings.dat', 
        names=['movieID','title','genre'], 
        engine='python', delimiter='::')


Next we'll implement our ratings matrix with our rows as movies and columns as users. After we'll have to normalize the matrix.

In [8]:
ratings = np.ndarray(
    shape=(np.max(ratings_data.movieID.values), np.max(ratings_data.userID.values)), 
    dtype=np.uint8)
ratings[ratings_data.movieID.values - 1, ratings_data.userID.values - 1] = ratings_data.rating.values

normalize_r = ratings - np.asarray([(np.mean(ratings, 1))]).T

SVD decomposition below

In [9]:
A = normalize_r.T / np.sqrt(ratings.shape[0] - 1)
U, S, V = np.linalg.svd(A)

Next we'll create a function for cosine similarity to recommend the most similar movies from list to the movie that was chosen and print the top 'n' recommended movies.

In [17]:
def top_cosine_similarity(ratings_data, movieID, top_n=10):
    index = movieID - 1 # Movie id starts from 1 in the dataset
    movie_row = ratings_data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratings_data, ratings_data))
    similarity = np.dot(movie_row, ratings_data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def print_recommended(movies_df, movieID, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movies_df[movies_df.movieID == movieID].title.values[0]))
    for id in top_indexes + 1:
        print(movies_df[movies_df.movieID == id].title.values[0])

In [19]:
k = 50
movieID = 1 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movieID, top_n)

print_recommended(movies_df, movieID, indexes)

Recommendations for 5: 

5
4
4
5
3
3
4
2
4
3


C:\Users\xxxdk\AppData\Local\Temp\ipykernel_12732\799633475.py:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(movie_row, ratings_data.T) / (magnitude[index] * magnitude)


something wrong above...